# Linear Regression with mini-batch

* y와 한 개 이상의 독립 변수 (또는 설명 변수) X와의 선형 상관 관계를 모델링하는 회귀분석 기법이다. 한 개의 설명 변수에 기반한 경우에는 단순 선형 회귀, 둘 이상의 설명 변수에 기반한 경우에는 다중 선형 회귀라고 한다. [참고: 위키피디아](https://ko.wikipedia.org/wiki/선형_회귀)

$$y_{\textrm{pred}} = \boldsymbol{W}^{\top}\boldsymbol{x} + b$$

* $\boldsymbol{x} = [x_{1}, x_{2}, \cdots, x_{d}]$
* $\boldsymbol{W} = [w_{1}, w_{2}, \cdots, w_{d}]$
* Loss function: $\mathcal{L} = \sum^{N} (y_{\textrm{pred}} - y)^{2}$

## Training Pseudo Code

```python
for epoch in max_epochs: # 1 epoch: 모든 데이터(N)를 한번 학습 시켰을 때
  for step in num_batches: # num_batches = int(data_size / batch_size)
    1. sampling mini-batches with batch_size
      1-1. data augmentation (필요하면)
    2. calculate logits
    3. calculate loss using logits and labels
    4. calculate the gradient with respect to weights
    5. update weights
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

tf.set_random_seed(219)
np.random.seed(219)

## Phase 1. Build a model

### Make data

In [ ]:
_a = 3
_b = -3
N = 200
data_x = np.random.uniform(low=0, high=5, size=N)
data_y = _a * data_x + _b + np.random.normal(0, 2, N)

plt.plot(data_x, data_y, 'ro')
plt.axhline(0, color='black', lw=1)
plt.axvline(0, color='black', lw=1)
plt.show()

### Create placeholders for inputs and labels

In [ ]:
# 여기를 직접 채워 넣으시면 됩니다.
# x, y는 배치 데이터를 받을 수 있는 placeholder
# x: inputs
x = tf.placeholder(tf.float32, name='x', shape=[None])

# y: labels
y = tf.placeholder(tf.int32, name='y', shape=[None])

### Create weight and bias

In [ ]:
# 여기를 직접 채워 넣으시면 됩니다.
# create Variables using `get_variable`
W = tf.get_variable(name='weight', initializer=tf.constant(0.0))
b = tf.get_variable(name='bias', shape=[],
                    initializer=tf.random_normal_initializer())

### Build a model: $y = Wx + b$

In [ ]:
# 여기를 직접 채워 넣으시면 됩니다.
with tf.variable_scope(name_or_scope='y_pred'):
  y_pred = W * x + b

### Define loss function

In [ ]:
# 여기를 직접 채워 넣으시면 됩니다.
# 이번에는 Hubber loss를 직접 구현해서 넣어 봅시다.
loss = tf.reduce_mean(tf.square(y - y_pred, name='loss'))

### Create a optimizer

In [ ]:
# 여기를 직접 채워 넣으시면 됩니다.
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

## Phase 2. Train a model

### Train a model

* stochastic gradient descent with mini-batch

In [ ]:
batch_size = 8

with tf.Session(config=sess_config) as sess:
  # Initialize all variables
  sess.run(tf.global_variables_initializer())
  
  writer = tf.summary.FileWriter('graphs/02_linear_reg_with_batch', sess.graph)
  writer.close()
  
  # train the model
  max_epochs = 100
  total_losses = []
  num_batches_per_epoch = int(N / batch_size)
  for epoch in range(max_epochs+1):
    total_loss = 0.0
    for step in range(num_batches_per_epoch):
      # 전체 데이터 N개중에 batch_size 만큼 데이터 random 하게 추출
      batch_index = np.random.choice(N, size=batch_size)
      x_ = data_x[batch_index]
      y_ = data_y[batch_index]
      # `sess.run` 부분을 직접 채워넣으시면 됩니다.
      _, loss_ = sess.run([train_op, loss],
                          feed_dict={x: x_, y: y_})
      total_loss += loss_ * batch_size
      
    total_loss /= float(N)
    total_losses.append(total_loss)
    
    if epoch % 10 == 0:
      print('Epoch {}: total_loss: {}'.format(epoch, total_loss))
      
  print('training done!')

  W_, b_ = sess.run([W, b])

### Print the results: W and b

* 정답 W = 3, b = -3

In [ ]:
print(W_, b_)

### Plot the loss funtion

In [ ]:
plt.plot(total_losses, label='total_loss')
plt.legend()
plt.show()

### Plot the results

In [ ]:
plt.plot(data_x, data_y, 'ro', label='Real data')
plt.plot(data_x, W_ * data_x + b_, lw=5, label='model')
plt.axhline(0, color='black', lw=1)
plt.axvline(0, color='black', lw=1)
plt.legend()
plt.show()